In [1]:
%%capture
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install accelerate==0.21.0


In [1]:
%%capture
!apt install git-lfs

In [2]:
from transformers.utils import send_example_telemetry

send_example_telemetry("multi_lingual_speech_recognition_notebook", framework="pytorch")

In [133]:
from datasets import load_dataset, load_metric, Audio
# Chia train test 80/20
common_voice_train = load_dataset("common_voice", "vi", split="train[:100]")
common_voice_test = load_dataset("common_voice", "vi", split="test[:20]")

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


In [134]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [135]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [136]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,Bên trong đèn hồng đèn đỏ đèn xanh lờ mờ luôn được bật sáng
1,Gần đây rất hay suy nghĩ mọi thứ quá nên
2,tiện đây tao nói cho mày nghe cái này nhé
3,Quân bước vội vào bên trong nhưng không thể bước được
4,bàn chân nào ở lại
5,ai cũng kêu đây là vụ án kinh hoàng nhất mà họ được thấy
6,Anh thích thì lát nữa tôi sẽ tặng anh
7,Ai là người chết đưa Nhi đến gặp thầy bói Miền
8,nên chúng em bị giữ lại phạt
9,Cũng hết thời gian cho kuman ăn


In [137]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [138]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [139]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,quân bước vội vào bên trong nhưng không thể bước được
1,họp lớp sau nhiều năm mỗi người một nơi
2,như không khí như màu xanh lá cỏ
3,tôi yêu việt nam
4,chinh lúc này mới giật mình nhìn sang bên cạnh
5,cũng chẳng biết là hợp thầy hợp thuốc nào
6,tiện đây tao nói cho mày nghe cái này nhé
7,được đặt ở khắp mọi miền việt nam
8,nếu thật thế thì tội nghiệp con bé lắm
9,lòng đen lòi cả con mắt


In [140]:
# def extract_all_chars(batch):
#   all_text = " ".join(batch["sentence"])
#   vocab = list(set(all_text))
#   return {"vocab": [vocab], "all_text": [all_text]}

In [141]:
# vocab_train = common_voice_train.map(
#   extract_all_chars, batched=True,
#   batch_size=-1, keep_in_memory=True,
#   remove_columns=common_voice_train.column_names
# )
# vocab_test = common_voice_test.map(
#   extract_all_chars, batched=True,
#   batch_size=-1, keep_in_memory=True,
#   remove_columns=common_voice_test.column_names
# )

In [142]:
# vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [143]:
# vocab_dict = {v: k for k, v in enumerate(vocab_list)}
# vocab_dict

In [144]:
# vocab_dict["|"] = vocab_dict[" "]
# del vocab_dict[" "]

In [145]:
# vocab_dict["[UNK]"] = len(vocab_dict)
# vocab_dict["[PAD]"] = len(vocab_dict)
# len(vocab_dict)

In [146]:
# import json
# with open('vocab.json', 'w') as vocab_file:
#     json.dump(vocab_dict, vocab_file)

In [147]:
# model_checkpoint = "facebook/wav2vec2-large-xlsr-53"
model_checkpoint = "phatjk/wav2vec2-large-vi"

In [148]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer_type = config.model_type if config.tokenizer_class is None else None
config = config if config.tokenizer_class is not None else None

In [149]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
  # "./",
  model_checkpoint,
  config=config,
  tokenizer_type=tokenizer_type,
  unk_token="[UNK]",
  pad_token="[PAD]",
  word_delimiter_token="|",
)

`use_fast` is set to `True` but the tokenizer class does not have a fast version.  Falling back to the slow version.


In [150]:
model_checkpoint_name = model_checkpoint.split("/")[-1]
repo_name = f"{model_checkpoint_name}-finetuining-vietnamese"

In [151]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 100
})

In [152]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [153]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=False, rate=16000)

tao chỉ muốn chơi với mày thôi 


In [154]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: tiện đây tao nói cho mày nghe cái này nhé 
Input array shape: (56064,)
Sampling rate: 16000


In [155]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

In [156]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [157]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [158]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [159]:
max_input_length_in_sec = 5.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [160]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [161]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [162]:
wer_metric = load_metric("wer")

In [163]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [164]:
from transformers import AutoModelForCTC

model = AutoModelForCTC.from_pretrained(
    model_checkpoint,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [165]:
if hasattr(model, "freeze_feature_extractor"):
  model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1914: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [166]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=32,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=200,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=50,
  eval_steps=50,
  logging_steps=50,
  learning_rate=3e-4,
  # warmup_steps=500,
  save_total_limit=2,
#   push_to_hub=True,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [167]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [168]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
50,0.512800,2.156235,0.971014
100,0.303000,2.200046,0.927536
150,0.201700,2.516819,0.949275
200,0.153900,2.720005,0.949275
250,0.103200,2.783602,0.978261
300,0.104900,2.825147,0.963768
350,0.090000,2.793265,0.978261
400,0.077000,2.819452,0.985507


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

TrainOutput(global_step=400, training_loss=0.19331276476383208, metrics={'train_runtime': 2114.1566, 'train_samples_per_second': 9.271, 'train_steps_per_second': 0.189, 'total_flos': 2.580686656565737e+18, 'train_loss': 0.19331276476383208, 'epoch': 200.0})

In [169]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'eval_loss': 2.8194515705108643,
 'eval_wer': 0.9855072463768116,
 'eval_runtime': 1.4986,
 'eval_samples_per_second': 13.346,
 'eval_steps_per_second': 2.002,
 'epoch': 200.0}

In [185]:
# Chon audio de test
import IPython.display as ipd
import numpy as np
import random
audio_num = random.randint(0, 100)
ipd.Audio(data=common_voice_train[audio_num]["input_values"], autoplay=False, rate=16000)

In [184]:
# Predict
from transformers import AutoModelForCTC

inputs = processor(common_voice_train[audio_num]["input_values"], sampling_rate=16000, return_tensors="pt")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
inputs = inputs.to(device)
with torch.no_grad():
    logits = model(**inputs).logits
    # Ket qua du doan
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    print("Result: ",transcription)

Result:  ['bàn chân nào ở lại']


In [179]:
import torch
torch.argmax(logits, dim=-1)

tensor([[45, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90,
         90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 36,
         90, 90, 90, 90, 90, 59, 46, 46, 69, 69, 69, 39, 50, 90, 90, 90, 90, 90,
         43, 69, 69, 69, 45, 86, 90, 80, 80, 69, 69, 46, 46, 40, 90, 90, 90, 90,
         36, 36, 69, 69, 82, 21, 80, 80, 69, 69, 54,  3, 90, 43, 43, 69, 69, 46,
         46,  4, 21, 71, 71, 90, 90, 90, 90, 69, 69, 69, 82, 42, 90, 90, 36, 36,
         69, 46,  3, 90, 90, 43, 43, 69, 69, 46, 46, 46, 21, 30, 30, 90, 90, 90,
         90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90,
         90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90,
         90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 69]], device='cuda:0')

Up file len hub

In [174]:
from huggingface_hub import notebook_login

notebook_login()

In [175]:
repo_name = "phatjk/wav2vec2-large-vi"
# tokenizer.push_to_hub(repo_name)
model.push_to_hub(repo_name)

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/phatjk/wav2vec2-large-vi/commit/7c9631c54209f1f8264ed6dac06e5d80d3589aa8', commit_message='Upload Wav2Vec2ForCTC', commit_description='', oid='7c9631c54209f1f8264ed6dac06e5d80d3589aa8', pr_url=None, pr_revision=None, pr_num=None)